In [1]:
import os
import sys
import glob
import time
import datetime
import numpy as np
import pandas as pd
import sqlite3

TODO: Use a config file to specify the required info (for final version)

From https://github.com/chenyz0601/mmd-project/tree/master/project1/PythonSrc

In [2]:
code_path = "./PythonSrc"
msd_subset_data_path = "./MillionSongSubset/data/"

In [3]:
sys.path.append(code_path)

In [4]:
import hdf5_getters as GETTERS

## Setup + Open DB

In [5]:
def encode_string(s):
    """
    Simple utility function to make sure a string is proper
    to be used in a SQLite query
    (different than posgtresql, no N to specify unicode)
    EXAMPLE:
      That's my boy! -> 'That''s my boy!'
    """
    return "'"+s.replace("'","''")+"'"

In [6]:
# Change this to your path to database
dbfile = "./MillionSongSubset/AdditionalFiles/subset_track_metadata.db"

In [7]:
conn = sqlite3.connect(dbfile)

In [8]:
# from that connection, get a cursor to do queries
c = conn.cursor()

# so there is no confusion, the table name is 'songs'
TABLENAME = 'songs'

## Demo of how to use the database

In [9]:
# list all tables in that dataset
# note that sqlite does the actual job when we call fetchall() or fetchone()
q = "SELECT name FROM sqlite_master WHERE type='table' ORDER BY name"
res = c.execute(q)
print("* tables contained in that SQLite file/database (should be only 'songs'):")
print(res.fetchall())

* tables contained in that SQLite file/database (should be only 'songs'):
[('songs',)]


In [10]:
# list all columns names from table 'songs'
q = "SELECT sql FROM sqlite_master WHERE tbl_name = 'songs' AND type = 'table'"
res = c.execute(q)
print('* get info on columns names (original table creation command):')
print(res.fetchall()[0][0])

* get info on columns names (original table creation command):
CREATE TABLE songs (track_id text PRIMARY KEY, title text, song_id text, release text, artist_id text, artist_mbid text, artist_name text, duration real, artist_familiarity real, artist_hotttnesss real, year int)


In [11]:
# list all indices
q = "SELECT name FROM sqlite_master WHERE type='index' AND tbl_name='songs' ORDER BY name"
res = c.execute(q)
print('* one of the index we added to the table to make things faster:')
print(res.fetchone())

* one of the index we added to the table to make things faster:
('idx_artist_id',)


In [12]:
# find the PRIMARY KEY of a query
# by default it's called ROWID, it would have been redefined if our primary key
# was of type INTEGER
q = "SELECT ROWID FROM songs WHERE artist_name='The Beatles'"
res = c.execute(q)
print('* get the primary key (row id) of one entry where the artist is The Beatles:')
print(res.fetchone())

* get the primary key (row id) of one entry where the artist is The Beatles:
(1040,)


In [13]:
# find an entry with The Beatles as artist_name
# returns all info (the full table row)
q = "SELECT * FROM songs WHERE artist_name='The Beatles' LIMIT 1"
res = c.execute(q)
print('* get all we have about one track from The Beatles:')
print(res.fetchone())

* get all we have about one track from The Beatles:
('TRAHSSO128EF347345', 'Derek Taylor - Introduction', 'SORTPSA12A67ADBFE2', 'Here There And Everywhere', 'AR6XZ861187FB4CECD', 'b10bbbfc-cf9e-42e0-be17-e2c3e1d2600d', 'The Beatles', 53.78567, 0.840409662154, 0.840462688027, 0)


## Create a pandas dataframe

In [14]:
q = "SELECT * FROM songs"
res = c.execute(q)

In [15]:
df = pd.DataFrame(res.fetchall(), columns=["Track ID", "Title", "Song ID", "Release", "Artist ID", "Artist MBIND", "Artist Name", "Duration", "Artist Familiarity", "Artist Hotness", "Year"])

In [16]:
df.head()

,Track ID,Title,Song ID,Release,Artist ID,Artist MBIND,Artist Name,Duration,Artist Familiarity,Artist Hotness,Year
0,TRACCVZ128F4291A8A,Deep Sea Creature,SOVLGJY12A8C13FBED,Call of the Mastodon,ARMQHX71187B9890D3,bc5e2ad6-0a4a-4d90-b911-e9a7e6861727,Mastodon,280.21506,0.780462,0.574275,2001
1,TRACCMH128F428E4CD,No Quieras Marcharte,SOGDQZK12A8C13F37C,Adelante,AR2PT4M1187FB55B1A,d54ea4a6-0b9c-4e47-bed0-289ae9ff4037,Los Chichos,191.68608,0.561590,0.420570,1984
2,TRACCSW128F148C7C3,If I...,SODMVJR12A6D4F985D,Ill Na Na,ARDI88R1187B98DAB2,fd87374e-ffde-4d36-89a8-8a073f795666,Foxy Brown,222.92853,0.687687,0.406686,0
3,TRACCXJ128F428F0CF,Werther - Lyric Drama in four Acts/Act I/Alors...,SOIWBDR12A8C13A4AC,Massenet: Werther,ARUKJUP12086C14589,8a5f2736-bcde-4a2e-8d50-72631d66a7ef,Ramón Vargas;Vladimir Jurowski,278.38649,0.391741,0.291265,0
4,TRACCVS12903D014F8,Ad Te Clamamus Exsvles Mortva Liberi,SOHCCIA12AC907577F,Pentagrammaton,ARZEWUR1187FB53DC8,0be59867-0da4-4e45-9b64-728cdf25487c,Enthroned,89.15546,0.593416,0.395710,2010


## How to use the getters for more song information

In [17]:
# signature is the raw value of each song
selectedFeatures = ["loudness", "tempo"]
num_features = len(selectedFeatures)
num_songs = 10
signature = np.zeros([num_songs, num_features])

In [18]:
# code from https://github.com/chenyz0601/mmd-project/blob/master/project1/Duplication.ipynb
s_counter = 0
songsIds = [0]*num_songs
for root, dirs, files in os.walk(msd_subset_data_path):
    files = glob.glob(os.path.join(root,'*'+'.h5'))
    for f in files :
        if s_counter == num_songs:
            break
        else:
            h5 = GETTERS.open_h5_file_read(f)
            f_counter = 0
            songsIds[s_counter] = GETTERS.get_song_id(h5)
            for features in selectedFeatures:
                temp = str(getattr(GETTERS, 'get_'+features)(h5))
                signature[s_counter,f_counter] = temp
                f_counter += 1
            s_counter += 1
            h5.close()

In [19]:
signature

array([[ -9.636, 124.059],
       [-11.061,  80.084],
       [-24.14 ,  54.874],
       [ -5.795,  77.15 ],
       [-16.477, 120.382],
       [-12.474,  99.024],
       [ -4.393, 175.673],
       [ -5.05 ,  87.999],
       [ -4.264,  92.897],
       [-13.885,  86.981]])

## Closing the connection (run when done)

In [20]:
# close the cursor and the connection
# (if for some reason you added stuff to the db or alter
#  a table, you need to also do a conn.commit())
c.close()
conn.close()